In [16]:
torch.__version__

'2.1.0+cu121'

In [21]:
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
import torchvision.models as models
from torch import optim
import torch.nn as nn
import torch.nn.functional as F
import json
import numpy as np
import os
import cv2
from tqdm import tqdm
import random

from matplotlib import pyplot as plt
import pandas as pd
from collections import namedtuple
from sklearn.metrics import accuracy_score
from copy import deepcopy

import albumentations as A
from albumentations.pytorch import ToTensorV2
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [22]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

In [40]:
BASE = '/mnt/d/Jupyter-Goodyoung/DiabetesSolution-AI'
SAVE_PATH = f"{BASE}/save"
MODEL_SAVE = f'{SAVE_PATH}/EfficientNetB0-2.pth' #
WORKERS = 4
EPOCHS = 50 # 훈련 epoch 지정
BATCH_SIZE = 8# batch size 지정

NUM_CLASSES = 722
RANDOM_STATE = 42 # seed 고정

DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
DEVICE = torch.device('cpu')
if not os.path.exists(SAVE_PATH):
    os.makedirs(SAVE_PATH)

seed_everything(RANDOM_STATE) # SEED 고정

In [41]:
import timm
model = timm.create_model('efficientnet_b0', pretrained=True)
model.classifier = nn.Linear(1280,NUM_CLASSES)

In [44]:
if os.path.exists(MODEL_SAVE):
    model.load_state_dict(torch.load(MODEL_SAVE, map_location=torch.device('cpu'))["model_state_dict"])
model.to(DEVICE)
model.eval()  # 모델을 평가 모드로 설정

EfficientNet(
  (conv_stem): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn1): BatchNormAct2d(
    32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
    (drop): Identity()
    (act): SiLU(inplace=True)
  )
  (blocks): Sequential(
    (0): Sequential(
      (0): DepthwiseSeparableConv(
        (conv_dw): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (bn1): BatchNormAct2d(
          32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): SiLU(inplace=True)
        )
        (se): SqueezeExcite(
          (conv_reduce): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
          (act1): SiLU(inplace=True)
          (conv_expand): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
          (gate): Sigmoid()
        )
        (conv_pw): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn2): BatchNormAct2d(
      

# Torch Mobile

In [47]:
import torch
import torchvision
from torch.utils.mobile_optimizer import optimize_for_mobile
from torch.jit.mobile import (
    _backport_for_mobile,
    _get_model_bytecode_version,
)

In [11]:
example = torch.rand(1, 3, 480, 480).to(device)
traced_script_module = torch.jit.trace(model, example)
traced_script_module_optimized = optimize_for_mobile(traced_script_module)
print('1')
traced_script_module_optimized._save_for_lite_interpreter("Efficient-Mobile_test.ptl",)

1


In [12]:
print("model version", _get_model_bytecode_version(f_input="Efficient-Mobile_test.ptl"))

_backport_for_mobile(f_input="Efficient-Mobile_test.ptl", f_output="Efficient-Mobile3.ptl", to_version=5)

print("new model version", _get_model_bytecode_version("Efficient-Mobile3.ptl"))

model version 8
new model version 5


In [36]:
print("new model version", _get_model_bytecode_version("Efficient-Mobile2.ptl"))

new model version 5
